# Tic-Tac-Toe Endgame
Para resolver o problema do tic-tac-toe endgame, usaremos como dataset 958 exemplos de formações finais de jogos, onde para as features existem três valores possíveis {x,o,b}, sendo _x_ representando um x no campo, _o_ representando um círculo e _b_ representando um campo vazio.
Por exemplo, se tivermos _X=[x,x,x,x,o,o,x,o,o]_, então teremos um campo da seguinte forma:
<br>xxx<br>
xoo<br>
xoo<br>
Nesse caso, o _x_ venceu e por iso, a sua classe é _positive_. Caso contrário, a sua classe é _negative_

## Importando e formatando o dataset
Para trabalharmos com o dataset, consideraremos que _[x,o,b]=[1,-1,0]_, e que _[positive,negative]=[1,-1]_

In [1]:
!pip install numpy
!pip install pandas
!pip install scikit-learn
!pip install plotly
!pip install scipy

In [2]:
import numpy as np
import pandas as pd

In [3]:
columns = [
    'top-left',
    'top-middle',
    'top-right',
    'middle-left',
    'middle-middle',
    'middle-right',
    'bottom-left',
    'bottom-middle',
    'bottom-right',
    'class'
]
df_ttt = pd.read_csv('../dataset/tic-tac-toe.data', names=columns)
df_ttt.head()

,top-left,top-middle,top-right,middle-left,middle-middle,middle-right,bottom-left,bottom-middle,bottom-right,class
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [4]:
X = df_ttt.drop(columns=['class'])
y = df_ttt['class']

In [5]:
def char_to_discrete(value: str) -> int:
    if value=='x':
        return 1
    elif value=='o':
        return -1
    elif value=='b':
        return 0
    else:
        return None

In [6]:
X = X.apply(lambda x : (x.apply(lambda y : char_to_discrete(y))))
y = y.apply(lambda x : 1 if x == 'positive' else 0)

In [7]:
X

,top-left,top-middle,top-right,middle-left,middle-middle,middle-right,bottom-left,bottom-middle,bottom-right
0,1,1,1,1,-1,-1,1,-1,-1
1,1,1,1,1,-1,-1,-1,1,-1
2,1,1,1,1,-1,-1,-1,-1,1
3,1,1,1,1,-1,-1,-1,0,0
4,1,1,1,1,-1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...
953,-1,1,1,1,-1,-1,-1,1,1
954,-1,1,-1,1,1,-1,1,-1,1
955,-1,1,-1,1,-1,1,1,-1,1
956,-1,1,-1,-1,1,1,1,-1,1


In [8]:
y

0      1
1      1
2      1
3      1
4      1
      ..
953    0
954    0
955    0
956    0
957    0
Name: class, Length: 958, dtype: int64

In [9]:
X = np.array(X)
y = np.array(y)

## Definindo a função de fitness
Para resolver o problema, teremos a seguinte construção:
$result=sigmoid(x\dot W)$, onde $x=[x_1,x_2,...,x_8,x_9]$ é o input e $W=[w_1,w_2,...,w_8,w_9]$ são os pesos, que serão multiplicados e então normalizados pela função $sigmoid$, que será usada para calcular o valor de $\hat{y}$, onde<br>
$\hat{y}=1, se\ result>0.5$<br>
$\hat{y}=0, se\ result<=0.5$<br>
Dessa forma, a fitness function será dada pela maximização de $sum(\hat{y}==y)$, ou seja, se:
$\hat{y}=[1,0,0,1,1]$ e $y=[1,1,0,1,1]$, $(\hat{y}==y)=[1,0,1,1,1]$, logo, $sum(\hat{y}==y)=4$

In [10]:
def sigmoid(x: np.ndarray) -> np.float64:
    return 1 / (1 + np.exp(-x))

def fitness_function(x: np.ndarray, W: np.ndarray, y: np.ndarray) -> np.ndarray:
    assert(x.shape[1]==W.shape[0])
    assert(x.shape[0]==y.shape[0])
    y_hat = sigmoid(x.dot(W))
    vfunc = np.vectorize(lambda x : 1 if x > 0.5 else 0)
    y_hat = vfunc(y_hat)
    
    return np.sum(np.apply_along_axis(lambda x : x==y, axis=0, arr=y_hat), axis=0)
    

In [11]:
x_dummy = np.random.randint(-1, 2, (20, 9))
W_dummy = np.random.uniform(-2, 2, (9, 10))
y_dummy = np.random.randint(0, 2, 20)

In [12]:
fitness_function(x_dummy, W_dummy, y_dummy)

array([10, 13,  7, 11, 11,  6, 14, 12,  9, 10])

## Distribuição do Dataset
Criação de K-Fold

## Definição dos hiperparâmetros

In [13]:
POP_SIZE       = 100       # Tamanho da população
PROB_CROSSOVER = 0.75      # Probabilidade de Crossing-over
PROB_MUTATION  = 0.5       # Probabilidade de Mutação
BOUNDS         = (-10, 10) # Limite de domínio
MUTATE_BOUND   = 0.01      # Porcentagem de quanto do domínio será mutacionado
ELITISM        = 0.2       # Porcentagem de quanto dos melhores indivíduos serão passados para a próxima geração
MAX_ITER       = 100       # Número máximo de gerações

In [14]:
gen = np.random.uniform(BOUNDS[0], BOUNDS[1], (X.shape[1], POP_SIZE))
print(gen.shape)

(9, 100)


In [15]:
bound_mut = MUTATE_BOUND * (np.abs(BOUNDS[0]) + np.abs(BOUNDS[1]))

In [75]:
for j in range(MAX_ITER):
    fitness = fitness_function(X, gen, y)
#     print(f'Geração {j}')
#     print(f'Média   {np.mean(fitness)}')
#     print(f'Melhor  {np.max(fitness)}')

    prob_distribution = fitness/np.sum(fitness)
    ind = np.array(range(0, POP_SIZE))
    temp = np.column_stack((prob_distribution, ind))
    temp = temp[np.argsort(temp[:,0]),]
    sorted_prob_distribution = temp[:,0]

    best_index = POP_SIZE - np.maximum(1, int(ELITISM*POP_SIZE))
    best_ind = range(best_index, POP_SIZE)
    best_ind = temp[best_ind, 1]

    selected1 = np.random.choice(np.array(range(len(sorted_prob_distribution))), p=sorted_prob_distribution, size=POP_SIZE)
    selected2 = np.random.choice(selected1, POP_SIZE, replace=False)

    ind      = np.asarray(temp[selected1, 1], dtype=int).tolist()
    selected = gen[:, ind]
    f1       = fitness[ind]
    ind      = np.asarray(temp[selected2, 1], dtype=int).tolist()
    mates    = gen[:, ind]
    f2       = fitness[ind]
    offspring = []
    r = np.random.uniform(0, 1, POP_SIZE)
    for i in range(0, POP_SIZE):
        if r[i] < PROB_CROSSOVER:
            gamma = np.random.uniform(0, 1, selected.shape[0])
            child = (1-gamma)*selected[:,i] + gamma*(mates[:,i])
        else:
            if f1[i] < f2[i]:
                child = mates[:,i]
            else:
                child = selected[:,i]
        offspring.append(child)

    r = np.random.uniform(0, 1, POP_SIZE)

    mutations = np.random.uniform(-bound_mut, bound_mut, (selected.shape[0], selected.shape[1]))
    r         = np.random.uniform(0, 1, (selected.shape[0], selected.shape[1])) 
    r         = r < PROB_MUTATION
    mutations = np.multiply(mutations, r)

    mutations_swapped = np.swapaxes(mutations, 0, 1)

    offspring = np.array(np.array(offspring) + mutations_swapped)
    offspring = offspring.reshape(offspring.shape[1], offspring.shape[0])

    new_fit = fitness_function(X, offspring, y)
    ind = range(0, POP_SIZE)
    temp = np.column_stack((new_fit, ind))
    temp = temp[np.argsort(-temp[:,0]),]

    ind_replace = np.array(temp[best_index:POP_SIZE, 1], dtype=int).tolist()

    next_gen = np.copy(offspring)

    next_gen[:,ind_replace] = gen[:,np.asarray(best_ind, dtype=int).tolist()]

    gen = np.copy(next_gen)


In [76]:
print(f'Accuracy {np.max(fitness)/X.shape[0]}')

Accuracy 0.8914405010438413


In [77]:
from sklearn.base import BaseEstimator
class GA(BaseEstimator):
    def __init__():
        pass